In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pymysql
import warnings
warnings.filterwarnings("ignore")

In [2]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [3]:
sql = '''
select 
tprm.order_create_time, tprm.model_number, tprm.order_id, tprm.total_receivable, too.pay_status, too.discount_money
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order_out too  on too.order_id=tprm.order_id;
'''
df = query(sql)
#  where pay_status = 2 and discount_money>0

In [4]:
df.loc[:, 'month'] = df.order_create_time.dt.strftime('%Y-%m')
df_new = df[(df.pay_status==2)&(df.discount_money>0)]
df_new_group = df_new.groupby('order_id').agg({'total_receivable': 'sum', 'discount_money': 'sum'})
df_new_group.loc[:, '使用买断优惠券比值'] = (df_new_group.discount_money/df_new_group.total_receivable).map(lambda x: format(x, '.2%'))
df_new_group_merge = df_new_group.merge(df_new[['order_id', 'month', 'model_number']], on='order_id', how='left')
df_new_group_merge.rename(columns={'order_id': '订单号', 'total_receivable': '总应收', 'discount_money': '使用买断优惠券金额', 'month': '月份', 'model_number': '型号'}, inplace=True)
df_new_group_merge[['月份', '订单号', '型号', '使用买断优惠券金额', '总应收', '使用买断优惠券比值']].to_excel('F:/需求/廖志彧需求/买断优惠券.xlsx', index=False)